In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import random
import glob
import os
from multiprocessing import Process
import sys
sys.path.append('..')
from modules import utils, constants

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Invalid MIT-MAGIC-COOKIE-1 key/home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/site-packages/stable_baselines/__init__.py:33: UserWarning: stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation.
  "stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation."


In [2]:
SEED = constants.SEED
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
tf.set_random_seed(SEED)
tf.compat.v1.set_random_seed(SEED)
SEED

42

In [3]:
constants.BETA, constants.CHECKPOINT_FREQ

(9, 1000000)

In [4]:
def run_dqn_model(model_type, steps):
    dir_name = f'seed_{SEED}_{steps}'
    parent_dir = f'../models/logs/{model_type}/missingness/0.1/no_step_penalty/update_freq_100'
    path = os.path.join(parent_dir, dir_name)
    os.mkdir(path)
  
    if model_type=='dqn':
        model = utils.stable_vanilla_dqn(X_train, y_train, steps, save=True, log_path=path, log_prefix='dqn', filename=f'dqn_{steps}')
    elif model_type=='ddqn':
        model = utils.stable_double_dqn(X_train, y_train, steps, save=True, log_path=path, log_prefix='ddqn', filename=f'ddqn_{steps}')
    elif model_type== 'dueling_dqn':
        model = utils.stable_dueling_dqn(X_train, y_train, steps, save=True, log_path=path, log_prefix='dueling_dqn', filename=f'dueling_dqn_{steps}')
    elif model_type == 'dueling_ddqn':
        model = utils.stable_dueling_ddqn(X_train, y_train, steps, save=True, log_path=path, log_prefix='dueling_ddqn', filename=f'dueling_ddqn_{steps}')
    elif model_type =='dqn_per':
        model = utils.stable_vanilla_dqn(X_train, y_train, steps, save=True, log_path=path, log_prefix='dqn_per', filename=f'dqn_per_{steps}', per=True)
    elif model_type == 'ddqn_per':
        model = utils.stable_double_dqn(X_train, y_train, steps, save=True, log_path=path, log_prefix='ddqn_per', filename=f'ddqn_per_{steps}', per=True)
    elif model_type == 'dueling_dqn_per':
        model = utils.stable_dueling_dqn(X_train, y_train, steps, save=True, log_path=path, log_prefix='dueling_dqn_per', filename=f'dueling_dqn_per_{steps}', per=True)
    elif model_type == 'dueling_ddqn_per':
        model = utils.stable_dueling_ddqn(X_train, y_train, steps, save=True, log_path=path, log_prefix='dueling_ddqn_per', filename=f'dueling_ddqn_per_{steps}', per=True)
    else:
        raise ValueError(f'Unknown model type - {model_type}!')
    return model

In [5]:
train_df = pd.read_csv('../new_data/train_set_missingness_0.1.csv')
train_df = train_df.fillna(-1)
train_df.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,proteinuria,biopsy_proven_lupus_nephritis,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label
0,1,1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
1,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1
2,0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0,...,1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,3.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,1


In [6]:
train_df.label.value_counts()

0    25240
1    25160
Name: label, dtype: int64

In [7]:
train_df.iloc[90]

ana                              0.0
fever                            0.0
leukopenia                       0.0
thrombocytopenia                 0.0
auto_immune_hemolysis            0.0
delirium                         0.0
psychosis                        0.0
seizure                          0.0
non_scarring_alopecia            0.0
oral_ulcers                      0.0
cutaneous_lupus                  0.0
pleural_effusion                 0.0
pericardial_effusion             0.0
acute_pericarditis               0.0
joint_involvement                0.0
proteinuria                      0.0
biopsy_proven_lupus_nephritis    0.0
anti_cardioliphin_antibodies     0.0
anti_β2gp1_antibodies            0.0
lupus_anti_coagulant             0.0
low_c3                           1.0
low_c4                           0.0
anti_dsdna_antibody              0.0
anti_smith_antibody             -1.0
label                            0.0
Name: 90, dtype: float64

In [8]:
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]
X_train, y_train = np.array(X_train), np.array(y_train)
X_train.shape, y_train.shape

((50400, 24), (50400,))

In [9]:
# model_names = ['dqn', 'ddqn', 'dueling_dqn', 'dueling_ddqn', 'dqn_per', 'ddqn_per', 'dueling_dqn_per', 
#                'dueling_ddqn_per']
model_names = ['dueling_dqn_per', 'dueling_ddqn_per']
procs = []
steps = int(100e6)

In [10]:
# run_dqn_model(model_names[0],steps)

In [11]:
for name in model_names:
#     run_dqn_model(name, steps)
    proc = Process(target=run_dqn_model, args=(name, steps))
    procs.append(proc)
    proc.start()

The environment seed is [42]
The environment seed is [42]








Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where












Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where










--------------------------------------
| % time spent exploring  | 95       |
| episodes                | 100000   |
| mean 100 episode reward | -0.6     |
| steps                   | 481650   |
| success rate            | 0.21     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 95       |
| episodes                | 100000   |
| mean 100 episode reward | -0.7     |
| steps                   | 481784   |
| success rate            | 0.17     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 90       |
| episodes                | 200000   |
| mean 100 episode reward | -0.7     |
| steps                   | 976497   |
| success rate            | 0.13     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 90       |
| episodes                | 200000   |
| mean 100 episode reward | -0.8     |
| steps              

--------------------------------------
| % time spent exploring  | 28       |
| episodes                | 1300000  |
| mean 100 episode reward | -0.8     |
| steps                   | 7530490  |
| success rate            | 0.08     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 27       |
| episodes                | 1600000  |
| mean 100 episode reward | -0.9     |
| steps                   | 7619205  |
| success rate            | 0.03     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 25       |
| episodes                | 1700000  |
| mean 100 episode reward | -0.8     |
| steps                   | 7846101  |
| success rate            | 0.08     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 23       |
| episodes                | 1800000  |
| mean 100 episode reward | -0.9     |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 1900000  |
| mean 100 episode reward | -0.8     |
| steps                   | 12765074 |
| success rate            | 0.08     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 4100000  |
| mean 100 episode reward | -0.8     |
| steps                   | 12661498 |
| success rate            | 0.1      |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 4200000  |
| mean 100 episode reward | -1       |
| steps                   | 12858271 |
| success rate            | 0.01     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 4300000  |
| mean 100 episode reward | -1       |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 6600000  |
| mean 100 episode reward | -1       |
| steps                   | 17672461 |
| success rate            | 0.01     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 6700000  |
| mean 100 episode reward | -0.9     |
| steps                   | 17875286 |
| success rate            | 0.07     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 2500000  |
| mean 100 episode reward | -0.9     |
| steps                   | 18208064 |
| success rate            | 0.07     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 6800000  |
| mean 100 episode reward | -1       |
| steps                  

...

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 27200000 |
| mean 100 episode reward | 0.6      |
| steps                   | 75044958 |
| success rate            | 0.81     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 9800000  |
| mean 100 episode reward | 0        |
| steps                   | 76423741 |
| success rate            | 0.5      |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 27300000 |
| mean 100 episode reward | 0.6      |
| steps                   | 75296058 |
| success rate            | 0.78     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 27400000 |
| mean 100 episode reward | -0.9     |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 29100000 |
| mean 100 episode reward | 0.6      |
| steps                   | 79891215 |
| success rate            | 0.8      |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 11000000 |
| mean 100 episode reward | 0.4      |
| steps                   | 81256528 |
| success rate            | 0.69     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 29200000 |
| mean 100 episode reward | 0.5      |
| steps                   | 80161221 |
| success rate            | 0.77     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 29300000 |
| mean 100 episode reward | 0.6      |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 31000000 |
| mean 100 episode reward | 0        |
| steps                   | 84711707 |
| success rate            | 0.52     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 12200000 |
| mean 100 episode reward | 0.1      |
| steps                   | 86026612 |
| success rate            | 0.55     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 31100000 |
| mean 100 episode reward | 0.4      |
| steps                   | 84971414 |
| success rate            | 0.68     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 12300000 |
| mean 100 episode reward | 0        |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 13400000 |
| mean 100 episode reward | 0.3      |
| steps                   | 90879081 |
| success rate            | 0.65     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 32900000 |
| mean 100 episode reward | 0.6      |
| steps                   | 89597011 |
| success rate            | 0.81     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 33000000 |
| mean 100 episode reward | 0.5      |
| steps                   | 89852982 |
| success rate            | 0.76     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 13500000 |
| mean 100 episode reward | 0.2      |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 34800000 |
| mean 100 episode reward | -0.5     |
| steps                   | 94540860 |
| success rate            | 0.23     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 14600000 |
| mean 100 episode reward | 0.4      |
| steps                   | 95960002 |
| success rate            | 0.71     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 34900000 |
| mean 100 episode reward | 0.3      |
| steps                   | 94807546 |
| success rate            | 0.64     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 14700000 |
| mean 100 episode reward | 0.4      |
| steps                  

In [12]:
for proc in procs:
    proc.join()
print('All jobs completed and terminated successfully')

All jobs completed and terminated successfully


In [13]:
# mylist = [{'name':30, 'list_feature': np.array([1,2,3]), 'label':0}, {'name':50, 'list_feature': np.array
#                                                                       ([4,5,6]), 'label':1}]
# a = pd.DataFrame(mylist)
# a

In [14]:
# a.iloc[0]['list_feature']

In [15]:
# from sklearn.ensemble import RandomForestClassifier
# X = a[['name', 'list_feature']]
# y = a['label']
# rf = RandomForestClassifier().fit(X, y)